Dependências 

In [11]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pandas as pd
from datetime import datetime

Funções Gerais

In [12]:
class GoogleSheetsAPI:
    def __init__(self):
        self.token_path = r"C:\Users\lucca.luna\Scripts\2 - Chaves\token_sheets.json"
        self.credentials_path = r"C:\Users\lucca.luna\Scripts\2 - Chaves\credentials_sheets.json"
        self.SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
        self.creds = None
        
        # Try to load credentials from token file
        if os.path.exists(self.token_path):
            self.creds = Credentials.from_authorized_user_file(self.token_path, self.SCOPES)

        # If credentials are not valid or do not exist, authenticate
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, self.SCOPES)
                self.creds = flow.run_local_server(port=0)

            # Save the credentials for the next run
            with open(self.token_path, "w") as token:
                token.write(self.creds.to_json())

    def sheets_to_dataframe(self, sheet_id, sheet_range):
        try:
            service = build("sheets", "v4", credentials=self.creds)

            sheet = service.spreadsheets()
            result = sheet.values().get(spreadsheetId=sheet_id, range=sheet_range).execute()
            values = result.get("values", [])

            if values:
                df = pd.DataFrame(values, columns=values[0]).reset_index(drop=True)
                return df
            else:
                print(f"No data found in {sheet_id}")
                return None
        except HttpError as err:
            print(err)

    def sheets_names(self, sheet_id):
        try:
            service = build("sheets", "v4", credentials=self.creds)

            # Obtém metadados da planilha
            sheet_metadata = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
            sheets = sheet_metadata.get('sheets', [])

            if sheets:
                df = pd.DataFrame(sheets).reset_index(drop=True)
                df = pd.json_normalize(df['properties'])
                df = df['title'].to_frame()
                df.columns.values[0] = "planilhas"
                return df
            else:
                return None
        except HttpError as err:
            print(err)

    def dataframe_to_sheets(self, df, sheet_id, sheet_range):
        try:
            service = build("sheets", "v4", credentials=self.creds)

            data = [df.columns.tolist()]  # Include header in the list of values
            data += df.values.tolist()    # Add DataFrame values

            body = {
                'values': data
            }
            result = service.spreadsheets().values().update(
                spreadsheetId=sheet_id, range=sheet_range,
                valueInputOption='RAW', body=body).execute()

            print('{0} cells updated.'.format(result.get('updatedCells')))
        except Exception as err:
            print(err)

class log:
    #Logging de execução do script
    def adicionar_termino():
        # Carregar o arquivo Excel como um DataFrame do Pandas
        arquivo_excel = r"C:\Users\lucca.luna\Scripts\0 - Log\Log.xlsx"
        df = pd.read_excel(arquivo_excel)

        # Adicionar a data e hora de término na última linha da coluna "Data de Término"
        ultima_linha = df.shape[0] - 1
        df.at[ultima_linha, "Data de Término"] = datetime.now()

        # Salvar o DataFrame de volta no arquivo Excel
        df.to_excel(arquivo_excel, index=False)

    def adicionar_inicio(nome):
        # Carregar o arquivo Excel como um DataFrame do Pandas
        arquivo_excel = r"C:\Users\lucca.luna\Scripts\0 - Log\Log.xlsx"
        # Altere para o caminho do seu arquivo
        df = pd.read_excel(arquivo_excel)

        # Criar um novo DataFrame com a nova linha
        nova_linha = pd.DataFrame([[nome, datetime.now()]], columns=["Nome", "Hora de Início"])

        # Concatenar o novo DataFrame com o DataFrame original
        df = pd.concat([df, nova_linha], ignore_index=True)

        # Salvar o DataFrame de volta no arquivo Excel
        df.to_excel(arquivo_excel, index=False)

Codigo

In [13]:
#Compilar dados de planejamento
Sheets = GoogleSheetsAPI()

sheet_id = "1AI7shZtkdDkc91sPxeU6bTzD0CwdDws5HI67RJ2iBbQ"

df_IRR_names = Sheets.sheets_names( sheet_id )

In [14]:
df_IRR_names = df_IRR_names[(df_IRR_names['planilhas'] != "MANUAL") & 
                            (df_IRR_names['planilhas'] != "LEADTIMES") & 
                            (df_IRR_names['planilhas'] != "Planejador x obra") & 
                            (df_IRR_names['planilhas'] != "MODELO") & 
                            (df_IRR_names['planilhas'] != "BASE DE DADOS") 
                            ]

In [15]:
df_IRR = pd.DataFrame()
for planilha in df_IRR_names['planilhas']:
    try:
        erro = "Extrair DF"
        sheet_id = "1AI7shZtkdDkc91sPxeU6bTzD0CwdDws5HI67RJ2iBbQ"
        sheet_range = f"'{planilha}'!B2:R"
        df = Sheets.sheets_to_dataframe(sheet_id, sheet_range)
        df = df.drop([0]).reset_index(drop=True)

        erro = "Retirar Brancos"
        df = df.loc[(df['Descrição da Restrição (Ação)'] != '') & (df['Descrição da Restrição (Ação)'] != 'CANCELADO')]
        non_null_columns = df.columns[df.columns.notna()]
        df = df[non_null_columns]


        erro = "Tratamento"
        # Substituir '-' por ':'
        df['Descrição da Restrição (Ação)'] = df['Descrição da Restrição (Ação)'].str.replace('-', ':')


        erro = "Dfs Vazios"
        try:
            df_na_vazio = df[df['N°'].isna() | (df['N°'] == '')]
            vazios = len(df_na_vazio)
        except:
            pass


        try:
            if vazios > 0:        
                df_na_vazio[['ID', 'Descrição da Restrição (Ação)']] = df_na_vazio['Descrição da Restrição (Ação)'].str.split(':', n=1, expand=True)
                df_na_preenchido = df[~df['N°'].isna() & (df['N°'] != '')]
                
                # Concatenar os DataFrames
                erro = "Concatenar Dados"
                df = pd.concat([df_na_vazio, df_na_preenchido], axis=0)

                erro = "Retirar DFs da Memoria"
                del df_na_vazio
                del df_na_preenchido
        except:
            pass    

        # Preenchendo a coluna 'ID' onde 'N°' está vazio
        erro = "Corrigir N"
        df['ID'] = ""
        df['ID'] = df.apply(lambda row: row['N°'] if pd.isna(row['ID']) or row['ID'] == "" else row['ID'], axis=1)

        df.drop(columns=['N°'], inplace=True)

        erro = "Criar Chave Obra"
        # Cria Coluna de chave de obra
        df['Chave Restrição'] = df['OBRA'].astype(str).str.cat(df['ID'].astype(str), sep=' - ')

        erro = "Filtrar Dados"
        #Filtra apenas as informações necessárias para serem enviadas
        df = df[((df['Setor Responsável']  == "Suprimentos") | (df['Setor Responsável']  == "Projeto") | (df['Setor Responsável']  == "Logistica")) & (df['Aprovação da restrição']  == "Aprovada")]

        erro = "Concatenar ao DF principal"
        df_IRR = pd.concat([df_IRR, df], ignore_index=True)
        del df
    except:
        print(f"{planilha} - {erro}")

C:\Users\lucca.luna\AppData\Local\Temp\ipykernel_15676\193294242.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na_vazio[['ID', 'Descrição da Restrição (Ação)']] = df_na_vazio['Descrição da Restrição (Ação)'].str.split(':', n=1, expand=True)
C:\Users\lucca.luna\AppData\Local\Temp\ipykernel_15676\193294242.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na_vazio[['ID', 'Descrição da Restrição (Ação)']] = df_na_vazio['Descrição da Restrição (Ação)'].str.split(':', n=1, expand=True)


In [16]:
# Obter lista de responsáveis únicos
df_responsaveis = df_IRR[['Responsável', 'Setor Responsável']].drop_duplicates().reset_index(drop=True)

# IDs e range da planilha
sheet_id = "1EcOx_tIukF4JjaTtRxHZu1nYjyac-hTt5uykL_V0-kA"
sheet_range = "'Emails'!A:F"

df_compradores_sheets = Sheets.sheets_to_dataframe(sheet_id, sheet_range)
df_compradores_sheets = df_compradores_sheets.drop_duplicates(subset=['Responsável']).reset_index(drop=True)
df_resultado = pd.merge(df_compradores_sheets, df_responsaveis, how="outer", on=['Responsável'], suffixes=('', '_duplicado'))
df_resultado['Setor Responsável'] = df_resultado['Setor Responsável'].combine_first(df_resultado['Setor Responsável_duplicado'])
df_resultado = df_resultado.drop(columns=['Setor Responsável_duplicado'])
df_resultado = df_resultado.drop_duplicates(subset=['Responsável']).reset_index(drop=True)
df_resultado = df_resultado[df_resultado['Responsável'] != 'Responsável']

# Remover valores NaN
df_resultado = df_resultado.fillna('')

# Enviar dados atualizados para a planilha
Sheets.dataframe_to_sheets(df_resultado, sheet_id, sheet_range)

625 cells updated.


In [17]:
#Merge das Planilhas para envio
df_IRR = pd.merge(df_IRR, df_resultado, how="left", on=["Responsável"])

#Criar Coluna de tempo restante
df_IRR['Data de Criação'] = pd.to_datetime(df_IRR['Data de Criação'], dayfirst=True, format='mixed')
df_IRR['Data Limite'] = pd.to_datetime(df_IRR['Data Limite'], dayfirst=True, format='mixed')

data_atual = datetime.now()

df_IRR['Dias Restantes'] = (df_IRR['Data Limite'] - data_atual).dt.days

In [18]:
#Fru-Fru visual
df_IRR = df_IRR.rename({'Descrição da Restrição (Ação)': 'Descrição', 'OBRA': 'Obra', 'STATUS': 'Status',}, axis = 1)
df_IRR['Data de Criação'] = df_IRR['Data de Criação'].dt.strftime('%d/%m/%Y')
df_IRR['Data Limite'] = df_IRR['Data Limite'].dt.strftime('%d/%m/%Y')
df_IRR = df_IRR.sort_values(by='Dias Restantes', ascending=True)

In [21]:
#Df Final
df_IRR = df_IRR[['Coordenador','Email','Responsável','ID','Obra','Categoria','Status','Descrição','Data de Criação','Data Limite','Dias Restantes']]
df_IRR.to_excel(r"C:\Users\lucca.luna\Scripts\4 - Scripts\0 - Sobre demanda\Newsletter\IRR.xlsx", index=False)
print(f"Finalizado {datetime.now()}")

Finalizado 2024-08-19 16:12:47.675041
